<a href="https://colab.research.google.com/github/Yogesh7920/DDOS-Detection/blob/master/DdosDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy

import numpy as np
import pandas as pd
import seaborn as sns
import os
import requests
import shutil
import random
import json
import pprint
from tqdm import tqdm
import re
import statistics 
from collections import defaultdict as dd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
import pickle
import time
import multiprocessing as mp
from joblib import Parallel, delayed
from ast import literal_eval 
from distutils.dir_util import copy_tree
from scapy.all import *
import time
from collections import Counter
from scapy.all import *
from scapy.layers.inet import IP
from scapy.layers.l2 import Ether
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
!wget https://mettl-miscellaneous-public.s3.ap-south-1.amazonaws.com/client_public_data/369004/ddos/Ddos_Detection_Dataset.7z -P /content/drive/My\ Drive/DDOS_Detection/Dataset
!wget https://mettl-miscellaneous-public.s3.ap-south-1.amazonaws.com/client_public_data/369004/ddos/Ddos_Detection_Dataset_Part2.7z -P /content/drive/My\ Drive/DDOS_Detection/Dataset

In [ ]:
!7za e /content/drive/My\ Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset.7z -o/content/drive/My\ Drive/DDOS_Detection/Dataset
!7za e /content/drive/My\ Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset.7z -o/content/drive/My\ Drive/DDOS_Detection/Dataset

In [ ]:
def get_packet_layers(packet):
    counter = 0
    while True:
        layer = packet.getlayer(counter)
        if layer is None:
            break

        yield layer.name
        counter += 1


def UDPCheck(pkt):
  s = set()
  for layer in get_packet_layers(pkt):
    s.add(layer)
  return 'UDP' in s


def getData(pcap):
    count = 0
    udp = 0
    times = []
    src = []
    dst = []
    size = []
    traffic = dd(lambda: 0)
    for (pkt_data, pkt_metadata) in tqdm(RawPcapReader(pcap)):
        count += 1
        ether = Ether(pkt_data)
        ip = IP(pkt_data)
        if UDPCheck(ether):
            udp += 1

        times.append(ip.time)
        src.append(ip.src)
        dst.append(ip.dst)
        size.append(ip.len)
        traffic[(ip.src, ip.dst)] += ip.len

    udp = (udp / count) * 100
    traffic = dict(traffic)

    result = {
        'times': times,
        'src': src,
        'dst': dst,
        'size': size,
        'traffic': traffic,
        'udp': udp,
        'count': count
    }
    return result

In [ ]:
def fix_digit(x):
    if len(x) == 1:
        return '0'+x
    else:
        return x


def get_feat_dir(dataset):
    feat = os.path.join(dataset, 'attack_feat')
    if not os.path.exists(feat):
        os.mkdir(feat)

    return feat


def get_dataset(dataset):
    attack = os.path.join(dataset, 'Ddos_Dataset_Part2')
    dirs = os.listdir(attack)
    dirs = list(map(lambda x: int(x), dirs))
    dirs.sort()
    dirs = filter(lambda x: x < 85, dirs)
    dirs = list(map(lambda x: str(x), dirs))
    dirs = list(map(fix_digit, dirs))

    completed = os.listdir(get_feat_dir(dataset))
    completed = set(list(map(lambda x: x.split('.')[0], completed)))
    dirs = filter(lambda x: x not in completed, dirs)

    dirs = list(map(lambda x: os.path.join(attack, x), dirs))
    return dirs

In [10]:
def get_feats(dataset_path,out_path, file, i):
    if os.isdir(out_path) == False:
      os.mkdir(out_path)
    files = os.listdir(dataset_path)
    start_time = time.time()
    print("current file %s --- %d of %d"%(file, i + 1, len(files)))
    write_file = file + ".p"
    if os.path.isfile(os.path.join(out_path, write_file)) == True:
        return "file completed"
    file_path = os.path.join(dataset_path, file)
    result = getData(os.path.join(dataset_path, file))
    with open(os.path.join(out_path, write_file), "wb") as f:
        pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("completed file %s ---- in %f minutes"%(file, (time.time() - start_time)/60))
    return result

In [ ]:
num_cores = multiprocessing.cpu_count()


def train(dataset_path, out_path):
  files = os.listdir(dataset_path)
  processed_list = Parallel(n_jobs=num_cores, backend="multiprocessing")(delayed(get_feats)(dataset_path, out_path, file, i) for i, file in enumerate(files))

# dataset-1
# benign1 
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/Ddos_benign/benign1"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/benign_feat/benign1"
train(dataset_path, out_path)
# benign2
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/Ddos_benign/benign2"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/benign_feat/benign2"
train(dataset_path, out_path)
# benign3
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/Ddos_benign/benign3"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/benign_feat/benign3"
train(dataset_path, out_path)
# p2pbox1
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/Ddos_benign/p2pbox1"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/benign_feat/p2pbox1"
train(dataset_path, out_path)

#attack
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/Ddos_Attack_data"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset/attack_feat"
train(dataset_path, out_path)



#dataset-2
#benign
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset_Part2/Ddos_benign"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset_Part2/benign_feat"
train(dataset_path, out_path)

#attack
dataset_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset_Part2/Ddos_Dataset_Part2"
out_path = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset_Part2/attack_feat"
train(dataset_path, out_path)

In [ ]:
dataset_1 = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset"
dataset_2 = "/content/drive/My Drive/DDOS_Detection/Dataset/Ddos_Detection_Dataset_Part2"

attack_1 = os.path.join(dataset_1, "attack_feat")
attack_2 = os.path.join(dataset_2, "attack_feat")
benign_1 = os.path.join(dataset_1, "benign_feat")
benign_2 = os.path.join(dataset_2, "benign_feat")

In [ ]:
for folder in os.listdir(benign_1):
  for file in os.listdir(os.path.join(benign_1, folder)):
    path = os.path.join(benign_1, folder)
    shutil.move(os.path.join(path, file), benign_1)

In [ ]:
attacks = os.listdir(attack_1) + os.listdir(attack_2)
benigns = os.listdir(benign_1) + os.listdir(benign_2)

len(attacks), len(benigns)

In [ ]:
i = 1
for x in os.listdir(attack_1):
  new_name = str(i) + ".p"
  i += 1
  os.rename(os.path.join(attack_1, x), os.path.join(attack_1, new_name))

for x in os.listdir(attack_2):
  new_name = str(i) + ".p"
  i += 1
  os.rename(os.path.join(attack_2, x), os.path.join(attack_2, new_name))

i = 1

for x in os.listdir(benign_1):
  new_name = str(i) + ".p"
  i += 1
  os.rename(os.path.join(benign_1, x), os.path.join(benign_1, new_name))

for x in os.listdir(benign_2):
  new_name = str(i) + ".p"
  i += 1
  os.rename(os.path.join(benign_2, x), os.path.join(benign_2, new_name))

In [ ]:
train_path = "/content/drive/My Drive/DDOS_Detection/Train_Data"
attack_path = os.path.join(train_path, "attack")
benign_path = os.path.join(train_path, "benign")

if os.path.isdir(attack_path) == False:
  os.mkdir(attack_path)
if os.path.isdir(benign_path) == False:
  os.mkdir(benign_path)

def move_files(src, dest):
  for file in os.listdir(src):
    try:
      shutil.copy(os.path.join(src, file), dest)
    except:
      #dont move ipynb checkpoints
      pass

In [ ]:
move_files(attack_1, attack_path)
move_files(attack_2, attack_path)
move_files(benign_1, benign_path)
move_files(benign_2, benign_path)

In [ ]:
head = [
  'freq_mean', 'freq_std', 'time_taken', 'bandwidth', 'traffic_mean', 'traffic_std','traffic_src',
  'traffic_dst', 'size_mean', 'size_std', 'size_max', 'size_min', 'size_total',
  'ip_src_count', 'ip_dst_count', 'ip_dst_mean', 'ip_dst_std', 'udp', 'attack'
]


def cumsub(times):
    result = []
    for t in range(len(times)-1):
        result.append(times[t+1]-times[t])

    return np.array(result)

def frequency(times):
    cb = cumsub(times)
    d = {
        'mean': np.mean(cb),
        'std': np.std(cb)
    }
    return d

def total_time(times):
    low, high = min(times), max(times)
    dt = high-low
    return dt

def traffic(times, traffic):
    dt = total_time(times)
    vals = np.array(list(traffic.values()))
    max_val = np.max(vals)
    src, dst = max(traffic, key = traffic.get)
    d = {
      'src': src,
      'dst':dst,
      'bandwidth': max_val/dt,
      'mean': np.mean(vals),
      'std': np.std(vals)
    }
    return d

def packetsize(size):
    size = np.array(size)
    d = {
      'mean': np.mean(size),
      'std': np.std(size),
      'max': np.max(size),
      'min': np.min(size),
      'total': np.sum(size)
    }
    return d

def unique_IP(src, dst):
    counter = Counter(dst)
    dst_counts = np.array(list(dict(counter).values()))
    d = {
      'src_count': len(set(src)),
      'dst_count': len(set(dst)),
      'mean_dst': np.mean(dst_counts),
      'std_dst': np.std(dst_counts)
    }
    return d

def features(d):
    freq = frequency(d['times'])
    time_taken = total_time(d['times'])
    traf = traffic(d['times'], d['traffic'])
    size = packetsize(d['size'])
    ip = unique_IP(d['src'], d['dst'])
    udp = d['udp']

    result = [
        freq['mean'], freq['std'], time_taken, traf['bandwidth'], traf['mean'], traf['std'],
        traf['src'],traf['dst'],size['mean'], size['std'], size['max'], size['min'], size['total'],
        ip['src_count'], ip['dst_count'], ip['mean_dst'], ip['std_dst'], udp
    ]

    return result

In [ ]:
dataset_feat = []
for file in tqdm(os.listdir(attack_path)):
    if file != '.ipynb_checkpoints':
        path = os.path.join(attack_path, file)
        attack = 1
        try:
            with open(path, "rb") as f:
                d = pickle.load(f)
        except:
            print(file)
            continue
        feat = features(d)
        feat.append(attack)
        dataset_feat.append(feat)

for file in tqdm(os.listdir(benign_path)):
    if file != '.ipynb_checkpoints':
        path = os.path.join(benign_path, file)
        attack = 0
        try:
            with open(path, "rb") as f:
                d = pickle.load(f)
        except:
            print(file)
            continue
        feat = features(d)
        feat.append(attack)
        dataset_feat.append(feat)

In [ ]:
dataset = pd.DataFrame(dataset_feat, columns=head)
X = dataset.drop(['attack', 'traffic_src', 'traffic_dst'], axis=1)
y = dataset['attack']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


knn = KNeighborsClassifier()
knn.fit(X_train, y_train) 
preds = knn.predict(X_test)
print(classification_report(y_test, preds))

In [ ]:
store_data = "/content/drive/My Drive/DDOS_Detection"

with open(os.path.join(store_path, "data.p"), "wb") as f:
    res = (knn,scaler, df)
    pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
dataset.to_csv("preprocessed_data.csv", index=False)